# Retrieving calibration data from Quantum Inspire

Pieter Eendebak <pieter.eendebak@tno.nl>

- Intro ...
- Compilation and transpilation options not available at this moment. See knowledge base for details
- Calibrations are coupled to experiments. Even though the calibrations are done on a periodic basis (e.g. once per day), every experiment gets mapped to such a calibration.
- Calibrations are backend specific

In [5]:
import os

import numpy as np
import setuptools
from qiskit import execute
from qiskit.circuit import ClassicalRegister, QuantumCircuit, QuantumRegister
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import get_token_authentication, load_account
from quantuminspire.qiskit import QI
from typing import Any

try:
    from rich import print as rprint
except ImportError:
    rprint = print

## Set credentials

In [14]:
QI_URL = r'https://api.quantum-inspire.com'
token = load_account()
authentication = get_token_authentication(token)
qi_api = QuantumInspireAPI(QI_URL, authentication)

QI.set_authentication(authentication, QI_URL)

## Run an experiment to generate data

In [11]:
backend_name = 'Spin-2'
if qi_api.get_backend_type_by_name(backend_name)["status"] == "OFFLINE":
        backend_name='Starmon-5'
qi_backend = QI.get_backend(backend_name)

print(f'selected backend {backend_name}')

circuit = QuantumCircuit(2)

circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()

qi_job = execute(circuit, backend=qi_backend, shots=256)
print(f'execute job on {qi_backend}')
qi_result = qi_job.result()
histogram = qi_result.get_counts(circuit)
print(f'Measured: {histogram}')

selected backend Starmon-5
execute job on Starmon-5

State	Counts
00		145
01		9
10		8
11		94


[None, None, None, None]

In [15]:
def remove_ordered_dict(d: Any) -> Any:
    """ Replace OrderedDict with dict """
    if isinstance(d, dict):
        return {k: remove_ordered_dict(v) for k, v in d.items()}
    else:
        return d
    
backend_name = 'Spin-2'
qi_backend = QI.get_backend(backend_name)
jj=qi_api.get_jobs()
backends = qi_api._get('https://api.quantum-inspire.com/backends')
backend_types =  qi_api.get_backend_types() 

backend_url=[b for b in backends if b['name']==backend_name][0]['url']
jobs = [j for j in jj if j['backend']==backend_url]
print(f'found {len(jj)} jobs')

if len(jobs)==0:
    raise Exception('you have not executed jobs in {backend_name}, calibration unavailable')
job=jobs[0]
print(f"found job with id {job['id']}")
result=qi_api.get_result_from_job(job['id'])
calibration=qi_api.get_calibration_from_result(result['id'])
calibration=remove_ordered_dict(calibration)
date=calibration['parameters']['system']['last_calibration_date']['value']
print(f'last calibration at {date}')

found 1875 jobs
found job with id 7064691
last calibration at 2021-06-25T09:48:26.546612


In [16]:
class Calibration():
    
    # TODO: type annotations
    def __init__(self, calibration_dictionary):
        
        self._calibration  = calibration_dictionary
        
        
    @property
    def calibration(self):
        return self._calibration
    
    def show_system_parameters(self):
        calibration=self._calibration
        system=calibration['parameters']['system']
        for key, item in system.items():
            pass
            if isinstance(item, dict):
                #print(dict(item))
                if np.all([k in item for k in ['unit', 'value'] ]):
                    value = item['value']
                    unit  = item['unit']
                    rprint(f'{key}: {value} [{unit}]')
                else:
                    timestamp = item['timestamp']
                    print(f'{key}: calibration at {timestamp}')
            else:
                print('unknown structure {key}')

    def show_qubit_parameters(self):
        calibration=self.calibration
        qdata=calibration['parameters']['qubits']
        for qubit, data in qdata.items():
                rprint(f'[green]Qubit[/green]: {qubit}')
                
                for key, item in data.items():
                    # TODO add exception handling
                    if isinstance(item, dict):
                        #print(dict(item))
                        symbol = key 
                        if 'symbol' in item:
                            symbol=item['symbol']
                            
                        if np.all([k in item for k in ['unit', 'value'] ]):
                            value = item['value']
                            unit  = item['unit']
                            print(f'  {symbol}: {value} [{unit}]')
                        else:
                            print('data at {key}?')
                    else:
                        print('unknown structure {key}')
                    
    def get_calibration(self, name):
        if name not in calibration['parameters']['system']:
            raise IndexError(f'calibration field {name} does not exist')
        c=calibration['parameters']['system'][name]
        return c
        
    def show_calibration(self, name):
        c=self.get_calibration( name)
            
        if name not in calibration['parameters']['system']:
            raise IndexError(f'error message {name}')
        c=calibration['parameters']['system'][name]
        rprint(f'[green]Calibration[/green]: {name}')
        simple_keys = ['timestamp', 'status']
        for key in simple_keys:
            try:
                value=c[key]
                rprint(f'    [green]{key}[/green]: {value}')
                #c['timestamp']
            except Exception as ex:
                print(f'{key} something sensible {ex}')
                
        fields = set(c.keys())-  set(simple_keys)
        rprint(f'    [green]other fields[/green]:', end='')
        print(f' {str(fields)}')

        return c
    
    def __repr__(self):
        return 'Calibration: {url} {timestamp} TODO: some nice representation'
    

In [18]:
s= Calibration(calibration)
s.show_system_parameters()
s.show_qubit_parameters()

try:
    c=s.show_calibration('readout_error_calibration')
except IndexError:
        print('readout error calibration does not exist')

fridge_temperature: 0.0296 [K]

last_calibration_date: 2021-06-25T09:48:26.546612

Qubit: q0

  ω: 15582000000.0 [Hz]
  T2*: 4.7e-06 [s]
  Vis: 78.3 [%]


Qubit: q1

  ω: 15799000000.0 [Hz]
  T2*: 9.8e-06 [s]
  Vis: 69.19999999999999 [%]
readout error calibration does not exist


In [ ]:
# TODO: pass standard deviations in calibration values (the decimals field is not suitable)